# SAR Dataset Accessibility

### Work in progress notebook

There are several ways to find and access the SAR datasets. These are the datasets which contain the calibrated geophysical range Doppler frequency shift retrievals from the ENVISAT ASAR wide-swath acqusitions obtained between 2002 and 2012. In the following a description of some selected methods for finding and extracting these datasets are presented.

### Find Data Through Web Search


All data is freely available and can be found in the MET Norway thredds catalog: https://thredds.met.no/thredds/catalog.html

![Thredds Dataset Overview](../images/Thredds_Dataset_Overview_image_cropped.png)

The ENVISAT ASAR datasets are located at: https://thredds.met.no/thredds/catalog/remotesensingenvisat/asar-doppler/catalog.html

Or just following this folder structure: Observations/Remotesensing_archive/ENVISAT_ASAR_Doppler:

![ENVISAT ASAR Doppler Overview](../images/ENVISAT_ASAR_Doppler_Overview_cropped.png)

Entering the subfolder, the separate netCDF-files are found under separate pathways dependent on their respective dates. Wanting to access the files for a specific date the following structure is followed: YEAR/MONTH/DAY

Underneath the path to 2012/01/27 is shown:

![ASAR 2012 overview](../images/ASAR_2012_overview_cropped.png)

![ASAR 2012/01 overview](../images/ASAR_2012_01_overview_cropped.png)

![ASAR 2012/01/27 overview](../images/ASAR_2012_01_27_overview_cropped.png)

The entire list of files from the specified date are then accessible (the list goes on).

### Access Data

Upon accessing a specific netCDF-file four different "Access"-options are available. These are "OPENDAP", "HTTPServer", "WCS" and "WMS". 

![ASAR 2012-01-27 netCDF overview.png](../images/ASAR_2012_01_27_netCDF_overview.png)

The separate options are explained in the following. In these examples the netCDF file "ASA_WSDV2PRNMI20120127_215005_000614583111_00101_51839_0000.nc" (the uppermost file under 2012/01/27) is used as an example.

#### OPENDAP - Using xarray:

The data is easily accessed through OPENDAP by the use of the xarray python package. Below is a an example on how to use xarray to open and investigate a desired dataset.
This procedure makes it easy to inspect the Dimensions, Coordinates, Data Variables, Indexes and Attributes of the dataset in question. 

In [1]:
# Import the required package: xarray
import xarray as xr

# Providing the OPENDAP-url
OPENDAP_url = 'https://thredds.met.no/thredds/dodsC/remotesensingenvisat/asar-doppler/2012/01/27/ASA_WSDV2PRNMI20120127_215005_000612433111_00101_51839_0000.nc'

# Using xarray to open the dataset using the OPENDAP-url
ds = xr.open_dataset(OPENDAP_url)

# Investigating the data as an xarray.Dataset 
ds

<xarray.Dataset> Size: 39MB
Dimensions:             (y: 602, x: 851)
Dimensions without coordinates: y, x
Data variables: (12/19)
    sensor_view_angle   (y, x) float32 2MB ...
    incidence_angle     (y, x) float32 2MB ...
    sensor_azimuth      (y, x) float32 2MB ...
    sigma0_VV           (y, x) float32 2MB ...
    topographic_height  (y, x) float32 2MB ...
    valid_land_doppler  (y, x) float32 2MB ...
    ...                  ...
    u_range             (y, x) float32 2MB ...
    std_u_range         (y, x) float32 2MB ...
    wind_direction      (y, x) float32 2MB ...
    wind_speed          (y, x) float32 2MB ...
    longitude           (y, x) float32 2MB ...
    latitude            (y, x) float32 2MB ...
Attributes: (12/75)
    GDAL:                       GDAL 3.2.1, released 2020/12/29
    acknowledgement:            GSAR IDL pre-processing software was develope...
    azimuth_start_time:         2012-01-27T21:50:05.173597
    contributor_email:          jeong-won.park@kopri.re.kr, hjoh@norceresearc...
    contributor_institution:    Korea Polar Research Institute (KOPRI), NORCE...
    contributor_name:           Jeong-Won Park, Harald Johnsen, Geir Engen
    ...                         ...
    ysamplefreq_slc:            1711.9144385026736
    ysize:                      629
    ytime:                      2012-01-27T21:50:05.173597Z
    ytime_slc:                  2012-01-27T21:50:04.077748
    summary_no:                 Kalibrert geofysisk Dopplerskift fra ENVISAT ...
    title_no:                   Kalibrert geofysisk ENVISAT ASAR Dopplerskift...

### Find Data Through CSW (Catalog Service on the Web)

Having problems finding the datasets using CSW. I am able to find the datasets used as examples here https://github.com/metno/esa-coscaw-data-search, but not the Envisat ASAR data we want for this manual... (just at staging site for now I assume...) - Note for self - See csw_test.ipynb for code example!

### Get Parent Datasets and their Children (or Dataset Series in ISO 19115) with OGC CSW

MET Norway organises datasets in parent-child relationships. A parent can be a given model simulation like [Arome-Arctic deterministic](https://data.csw.met.no/?mode=opensearch&service=CSW&version=2.0.2&request=GetRecords&elementsetname=full&typenames=csw:Record&resulttype=results&q=deterministic), where the link provides the OGC CSW result of a search for "deterministic".

The same search but with results provided in ISO format: https://data.csw.met.no/csw?SERVICE=CSW&VERSION=2.0.2&REQUEST=GetRecords&RESULTTYPE=results&TYPENAMES=csw:Record&ElementSetName=full&q=deterministic&outputschema=http://www.isotc211.org/2005/gmd.

Here, a field gmd:parentIdentifier provides the metadata identification of the parent dataset, i.e., no.met:806070da-e9f3-4d03-ba1d-26b843961634.

Get the parent dataset:

     https://data.csw.met.no/csw?service=CSW&version=2.0.2&request=GetRepositoryItem&id=no.met:806070da-e9f3-4d03-ba1d-26b843961634.

Get all its children:

     https://data.csw.met.no/csw?SERVICE=CSW&VERSION=2.0.2&REQUEST=GetRecords&RESULTTYPE=results&TYPENAMES=csw:Record&ElementSetName=full&outputFormat=application%2Fxml&outputschema=http://www.isotc211.org/2005/gmd&CONSTRAINTLANGUAGE=CQL_TEXT&CONSTRAINT=apiso:ParentIdentifier%20like%20%27no.met:806070da-e9f3-4d03-ba1d-26b843961634%27.

To find all parent datasets:

     https://csw.s-enda-staging.k8s.met.no/csw?SERVICE=CSW&VERSION=2.0.2&REQUEST=GetRecords&RESULTTYPE=results&TYPENAMES=csw:Record&ElementSetName=full&outputschema=http://www.isotc211.org/2005/gmd&CONSTRAINTLANGUAGE=CQL_TEXT&CONSTRAINT=dc:type%20like%20%27series%27.


#### Find Data with OpenSearch

OpenSearch is a way for websites and search engines to publish search results in a standard and accessible format.

To find all datasets in the catalogue:

    https://data.csw.met.no/?mode=opensearch&service=CSW&version=2.0.2&request=GetRecords&elementsetname=full&typenames=csw:Record&resulttype=results

Or datasets within a given time span:

    http://data.csw.met.no/?mode=opensearch&service=CSW&version=2.0.2&request=GetRecords&elementsetname=full&typenames=csw:Record&resulttype=results&time=2000-01-01/2020-09-01

Or datasets within a geographical domain (defined as a box with parameters min_longitude, min_latitude, max_longitude, max_latitude):

    https://data.csw.met.no/?mode=opensearch&service=CSW&version=2.0.2&request=GetRecords&elementsetname=full&typenames=csw:Record&resulttype=results&bbox=0,40,10,60

Or, datasets with "arome-arctic 2.5Km deterministic" in the title:

    https://data.csw.met.no/?mode=opensearch&service=CSW&version=2.0.2&request=GetRecords&elementsetname=full&typenames=csw:Record&resulttype=results&q=arome-arctic\\%202.5Km\\%20deterministic



### More Advanced Geographical Search with OGC CSW

PyCSW opensearch only supports geographical searches querying for a box. For more advanced geographical searches, one must write specific XML files. For example:

* To find all datasets containing a point (my_xml_request_containing_a_point.xml):

```xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords
    xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
    xmlns:ogc="http://www.opengis.net/ogc"
    xmlns:gml="http://www.opengis.net/gml"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    service="CSW"
    version="2.0.2"
    resultType="results"
    maxRecords="10"
    outputFormat="application/xml"
    outputSchema="http://www.opengis.net/cat/csw/2.0.2"
    xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" >
  <csw:Query typeNames="csw:Record">
    <csw:ElementSetName>full</csw:ElementSetName>
    <csw:Constraint version="1.1.0">
      <ogc:Filter>
        <ogc:Contains>
          <ogc:PropertyName>ows:BoundingBox</ogc:PropertyName>
          <gml:Point>
            <gml:pos srsDimension="2">59.0 4.0</gml:pos>
          </gml:Point>
        </ogc:Contains>
      </ogc:Filter>
    </csw:Constraint>
  </csw:Query>
</csw:GetRecords>
```

* To find all datasets intersecting a polygon (my_xml_request_intersecting_a_polygon.xml):

```xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords
    xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
    xmlns:gml="http://www.opengis.net/gml"
    xmlns:ogc="http://www.opengis.net/ogc"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    service="CSW"
    version="2.0.2"
    resultType="results"
    maxRecords="10"
    outputFormat="application/xml"
    outputSchema="http://www.opengis.net/cat/csw/2.0.2"
    xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" >
  <csw:Query typeNames="csw:Record">
    <csw:ElementSetName>full</csw:ElementSetName>
    <csw:Constraint version="1.1.0">
      <ogc:Filter>
        <ogc:Intersects>
          <ogc:PropertyName>ows:BoundingBox</ogc:PropertyName>
          <gml:Polygon>
            <gml:exterior>
              <gml:LinearRing>
                <gml:posList>
                  47.00 -5.00 55.00 -5.00 55.00 20.00 47.00 20.00 47.00 -5.00
                </gml:posList>
              </gml:LinearRing>
            </gml:exterior>
          </gml:Polygon>
        </ogc:Intersects>
      </ogc:Filter>
    </csw:Constraint>
  </csw:Query>
</csw:GetRecords>
```

* To find all datasets intersecting a polygon within a given time span (my_xml_request_intersecting_a_polygon_within_a_given_time_span.xml):

```xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords
    xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
    xmlns:gml="http://www.opengis.net/gml"
    xmlns:ogc="http://www.opengis.net/ogc"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    service="CSW"
    version="2.0.2"
    resultType="results"
    maxRecords="100"
    outputFormat="application/xml"
    outputSchema="http://www.opengis.net/cat/csw/2.0.2"
    xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" >
  <csw:Query typeNames="csw:Record">
    <csw:ElementSetName>summary</csw:ElementSetName>
    <csw:Constraint version="1.1.0">
      <ogc:Filter>
        <ogc:And>
          <ogc:Intersects>
            <ogc:PropertyName>ows:BoundingBox</ogc:PropertyName>
            <gml:Polygon>
              <gml:exterior>
                <gml:LinearRing>
                  <gml:posList>
                    63.3984 7.65173 60.7546 5.0449 59.0639 10.187 62.9065 12.4944 63.3984 7.65173
                  </gml:posList>
                </gml:LinearRing>
              </gml:exterior>
            </gml:Polygon>
          </ogc:Intersects>
          <ogc:PropertyIsGreaterThanOrEqualTo>
            <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
            <ogc:Literal>2022-03-01 00:00</ogc:Literal>
          </ogc:PropertyIsGreaterThanOrEqualTo>
          <ogc:PropertyIsLessThanOrEqualTo>
            <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
            <ogc:Literal>2023-03-08 00:00</ogc:Literal>
          </ogc:PropertyIsLessThanOrEqualTo>
        </ogc:And>
      </ogc:Filter>
    </csw:Constraint>
  </csw:Query>
</csw:GetRecords>
```

* Then, you can query the CSW endpoint and print the response text using, e.g., python:

In [2]:
import requests

# Define the headers
headers = {'Content-Type': 'application/xml'}

# Specify the xml-file that should be used for the search
# my_xml_request = 'my_xml_request_containing_a_point.xml'
my_xml_request = 'my_xml_request_intersecting_a_polygon.xml'
# my_xml_request = 'my_xml_request_intersecting_a_polygon_within_a_given_time_span.xml'

# Open and read the XML file
with open(my_xml_request, 'r') as file:
    xml_data = file.read()

# Send the POST request
response = requests.post('https://data.csw.met.no', data=xml_data, headers=headers)

# The response text
print(response.text)

# Extract the OPENDAP urls
import xarray as xr
import re
import sys

# The pattern 'https.*?\.nc(?:ml)?' is "https://thredds.met.no/thredds/dodsC/{regardless_of_what_is_in_between}.ncml" 
# where the "ml" ending is inculded only if found.
my_pattern= r'https://thredds.met.no/thredds/dodsC/.*?\.nc(?:ml)?'


# findall() function returns all non-overlapping matches of my_pattern in string, as a list of strings
opendap_urls = re.findall(my_pattern, response.text)

# List of OPENDAP urls
print(f'List contains {len(opendap_urls)} urls:')
print(opendap_urls)

# Check if there are any files - Statement if not
if len(opendap_urls) > 0:

    # Open the first dataset in the list of urls
    ds = xr.open_dataset(opendap_urls[0])
    
else:
    ds = "No file(s) match the search criterias."

ds

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!-- pycsw 2.7.dev0 -->
<csw:GetRecordsResponse xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dct="http://purl.org/dc/terms/" xmlns:gmd="http://www.isotc211.org/2005/gmd" xmlns:gml="http://www.opengis.net/gml" xmlns:ows="http://www.opengis.net/ows" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2.0.2" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd"><csw:SearchStatus timestamp="2024-09-04T07:20:51Z"/><csw:SearchResults numberOfRecordsMatched="190524" numberOfRecordsReturned="10" nextRecord="11" recordSchema="http://www.opengis.net/cat/csw/2.0.2" elementSet="full"><csw:Record><dc:identifier>no.met:8195279b-1de4-484e-ac9c-2dba99a4da35</dc:identifier><dc:title>Metop-C AVHRR/3 level-1C direct broadcast data in satellite swath, 2021-11-26T07:39:57Z.</dc:title><dc

<xarray.Dataset> Size: 367MB
Dimensions:                              (y: 4077, x: 2048)
Coordinates:
    longitude                            (y, x) float32 33MB ...
    latitude                             (y, x) float32 33MB ...
Dimensions without coordinates: y, x
Data variables:
    CHANNEL_1                            (y, x) float32 33MB ...
    CHANNEL_2                            (y, x) float32 33MB ...
    CHANNEL_3a                           (y, x) float32 33MB ...
    CHANNEL_3b                           (y, x) float32 33MB ...
    CHANNEL_4                            (y, x) float32 33MB ...
    CHANNEL_5                            (y, x) float32 33MB ...
    sensor_zenith_angle                  (y, x) float32 33MB ...
    solar_zenith_angle                   (y, x) float32 33MB ...
    sun_sensor_azimuth_difference_angle  (y, x) float32 33MB ...
Attributes: (12/64)
    Conventions:                  CF-1.7, ACDD-1.3
    access_constraint:            Open
    ancillary_timeliness:         NRT
    collection:                   METNCS
    contributor_email:            post@met.no,post@met.no
    contributor_institution:      Norwegian Meteorological Institute,Norwegia...
    ...                           ...
    references:                   https://data.met.no/dataset/8195279b-1de4-4...
    related_dataset:              no.met:d42548cc-337f-4005-91f4-a5dc306244b0...
    summary_no:                   Direktesendte satellittdata mottatt ved Met...
    summary:                      Direct Broadcast satellite data received at...
    title:                        Metop-C AVHRR/3 level-1C direct broadcast d...
    title_no:                     Metop-C AVHRR/3 level-1C direktesendte sate...

### How to Visualize Data with WMS (Web Map Service)

#### By the Use of data.met.no

Using data.met.no it is possible to both find and visualise datasets. The web search interface can be accessed from the "Data Catalog" menu item, or directly at https://data.met.no/metsis/search (BUT I CANT FIND THE ENVISAT ASAR DATA HERE... Assuming that these are not available here as of yet (only on staging page?)). As seen below the search interface consists of a map and a series of filters.

![Data Catalog Overview](../images/DataMetNo_Data_Catalog_Overview_image.png)

The map provides a pagination of available datasets in the metadata catalog [max/min longitude/latitude rectangle], sorted to showcase the latest additions first. One can also interact with the map to better diplay the results, and to perform data search.

* "Select Projection" located just above the map can be altered to change the map projection. "Spatial filter" can be set to both "Within" and "Intersects".
* The "Create bounding box"-button enables to set a bounding box directly on the map and works as a filter on the results.
* The "Reset Search"-button clears the filters and starts a new search.
* The "Reset Map"-button resets the map.

Map widgets allows direct interaction with the map:

* +/-:                     Zoom in/out.
* E:                       Zooms to the extent of the displayed datasets.
* Menu tag:                Opens side panel where WMS Layers, Features and Base Layers can be altered.
* Magnifying glass:        Enables searching for location names.
* '>>':                    Showing the location in an overview world map.
* Upper right hand widget: Full screen mode

Search filters can also be used to find the desired datasets. The results are updated dynamically when filters are selected. These allows:

* A full text search block where the options "Contains all of these words" and "Contains any of these words" are eligible.
* Start and end date of the desired datasets.
* An option named "Has children" to determine whethere datasets are parents with children (i.e. records of the same type).
* The desired sorting mechanism.
* Isotopic categories: The general subjects for which the geospatial data may be relevant, as defined by the ISO standard.
* Keywords: Keywords from a controlled vocabulary.
* Activity type: The nature of the dataset(s) generation process (Numerical Simulation, Climate Indicator, In Situ Land-based station, Space Borne Instrument).
* Project: Datsets related to a certain project.

By clicking the "Reset"-button all filters are removed and a new search can be initiated.




#### By the Use of QGIS

As the MET Norway's S-ENDA CSW catalog service is also available through QGIS, datasets can be found and inspected as follows:

1. From the menu select "Web > MetaSearch > MetaSearch".
2. Select "Services > New".
3. For the name type "data.csw.met.no".
4. For the URl type "https://data.csw.met.no".


To get a list of the available datasets there is the option to add different search parameters under the "Search" tab. By further clicking at "Search" the available datasets which correspond to the different paramteres will be displayed. 